# Neo4j



### 安装Neo4j 
```
wget https://labfile.oss.aliyuncs.com/courses/1354/neo4j-community-3.5.1-unix.tar.gz
tar -axvf neo4j-community-3.5.1-unix.tar.gz

cd neo4j-community-3.5.1-unix/bin
./neo4j start   // 启动
# ./neo4j stop //停止

```
打开浏览器访问   http://localhost:7474/

### 远程打开 

需要修改文件conf/neo4j.conf

```
# Bolt connector
dbms.connector.bolt.enabled=true
#dbms.connector.bolt.tls_level=OPTIONAL
dbms.connector.bolt.listen_address=0.0.0.0:7687

# HTTP Connector. There can be zero or one HTTP connectors.
dbms.connector.http.enabled=true
dbms.connector.http.listen_address=0.0.0.0:7474
```

确保以下两个端口 7474  7687 可以访问

### 修改默认密码 
第一次访问默认账号 neo4j，默认密码 neo4j，输入密码点击 Connect 后会提示修改初始密码，



### 下载图数据库文件 

```
wget https://dikers-data.s3.cn-northwest-1.amazonaws.com.cn/dataset/graph.db.dump

neo4j stop
    
neo4j-admin load --from=graph.db.dump --database=graph.db --force  

neo4j start
```


在neo4j 上执行 
```

MATCH (n:`角色`) RETURN n LIMIT 25

```

# 命名实体识别

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from collections import Counter
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
print(keras.__version__)

### 样本数据

In [ ]:
raw_text = '''张无忌，金庸武侠小说《倚天屠龙记》人物角色，中土明教第三十四代教主。武当七侠之一张翠山与天鹰教紫微堂主殷素素之子，明教四大护教法王之一金毛狮王谢逊义子。
              张翠山，《倚天屠龙记》第一卷的男主角，在武当七侠之中排行第五，人称张五侠。与天鹰教殷素素结为夫妇，生下张无忌，后流落到北极冰海上的冰火岛，与谢逊相识并结为兄弟。
              殷素素，金庸武侠小说《倚天屠龙记》第一卷的女主人公。天鹰教紫薇堂堂主，容貌娇艳无伦，智计百出，亦正亦邪。与武当五侠张翠山同赴王盘山，结果被金毛狮王谢逊强行带走，三人辗转抵达冰火岛。殷素素与张翠山在岛上结为夫妇，并诞下一子张无忌。
              谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王”。
           '''
annotations = {'name':['张无忌','张翠山','殷素素','谢逊'], 'book':['倚天屠龙记'],'org':['明教','武当','天鹰教']}
raw_text, annotations

### 命名实体标注

```
张 name_B  //begin
无 name_M  //中间
忌 name_E  //end
```

In [ ]:
import re

# 先去掉原始文本中的换行和空格符
raw_text = raw_text.replace('\n', '').replace(' ', '')
# 初始化 label：将其全部初始化为 O
labels = len(raw_text)*['O']

# 通过 key-value 的方式遍历 annotations 字典，进行转换
for ann, entities in annotations.items():
    for entity in entities:
        # 先生成实体对应的 BME 标注类型
        B, M, E = [['{}_{}'.format(ann,i)] for i in ['B','M','E']]
        # 计算实体词中的数量
        M_len = len(entity) - 2
        # 生成 label，如果词中数为0，则直接为 BE，不然按数量添加 M
        label = B + M * M_len + E if M_len else B + E
        # 从原始文本中找到实体对应出现的所有位置
        idxs = [r.start() for r in re.finditer(entity, raw_text)]

        for idx in idxs:
        # 替换原 label 中的 O 为实际 label
            labels[idx:idx+len(entity)] = label


# 打印原始文本和对应转换后的 label
for ann,label in zip(raw_text[0:20],labels[0:20]):
    print(ann, label)

In [ ]:
# 统计每个字出现的次数
word_counts = Counter(raw_text)
# 建立字典表，只记录出现次数不小于 2 的字
vocab = [w for w, f in iter(word_counts.items()) if f >= 2]

In [ ]:
vocab[0:10]

In [ ]:
labels[0:20]

In [ ]:
label_set = list(set(labels))
label_set

In [ ]:
# 拆分训练集，每一句话作为一个样本，先找到每个句号的位置
sentence_len = [r.start()+1 for r in re.finditer('。', raw_text)]

# 进行拆分，这里要注意最后一个句号后面不需要拆分，所以最后一个位置不需要取到
split_text = np.split(list(raw_text), sentence_len[:-1])
split_label = np.split(labels, sentence_len[:-1])
split_text[0]

In [ ]:
split_label[0]

In [ ]:
# 构建词袋模型，这里要将字典从 2 开始编号，把 0 和 1 空出来，0 作为填充元素，1 作为不在字典中的字的编号
word2idx = dict((w,i+2) for i,w in enumerate(vocab))
label2idx = [[label_set.index(w) for w in s] for s in split_label]

In [ ]:
label2idx[0]

```
9是普通字符
3,  // 张   人物开始
8,  // 无    中间
0,  // 忌   人物结束


4,  作品开始
1,  中间
1,  中间
1,  中间
5,  作品结束


```

In [ ]:
# 构建输入，即对于样本中每一个字，从词袋模型中找到这个字对应的 idx，出现频率过低的字，并没有出现在词袋模型中，此时将这些字的 idx 取为 1
train_x = [[word2idx.get(w, 1) for w in s] for s in split_text]

max_len = 64

# 在输入的左边填充 0，在输出的左端填充-1
train_x = pad_sequences(train_x, max_len, value=0)
train_y = pad_sequences(label2idx, max_len, value=-1)
train_y = np.expand_dims(train_y, 2)


In [ ]:
print("x shape ", train_x.shape)
print("y shape ", train_y.shape)

In [ ]:
train_x[0]

In [ ]:
train_y[0]

In [ ]:
# 定义模型的超参
EMBED_DIM = 200
BiRNN_UNITS = 200

# 初始化模型
model = Sequential()
# 添加 Embedding 层，将输入转换成向量
model.add(Embedding(len(vocab)+2, EMBED_DIM, mask_zero=True))
# 添加 BiLstm 层
model.add(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True)))
# 初始化 crf
crf = CRF(len(train_y), sparse_target=True)
# 将 crf 添加到模型中
model.add(crf)
model.summary()
# 编译模型
model.compile('adam', loss=crf_loss, metrics=[crf.accuracy])

In [ ]:
model.fit(train_x, train_y, batch_size=9, epochs=500)
model.save('./output/ner_model.h5')

In [ ]:
text = '谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王"。'
text

In [ ]:
# 将预测数据转换为特征向量
pred_x = [word2idx.get(w, 1) for w in text]
pred_x = pad_sequences([pred_x], max_len)
pred_x

In [ ]:
# 使用模型进行预测
pred = model.predict(pred_x)
pred.shape

In [ ]:
# 去除多余的维度
pred = np.squeeze(pred)[-len(text):]
pred.shape

In [ ]:
# 把输出向量转换为 label 对应的 idx
result = [np.argmax(r) for r in pred]
result

In [ ]:
# 打印输出结果
reslut_labels = [label_set[i] for i in result]
for w, l in zip(text, reslut_labels):
    print(w, l)

# 关系抽取

In [ ]:
# 对于 lists 中每一个子列表，第一个元素为实体1，第二个元素为实体2，第三个元素为实体1对实体2的关系，第四个元素为文本。
lists = [['杨康','杨铁心','子女','杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
         ['杨康','杨铁心','子女','丘处机与杨铁心、郭啸天结识后，以勿忘“靖康之耻”替杨铁心的儿子杨康取名。'],
         ['杨铁心','包惜弱','配偶','金国六王爷完颜洪烈因为贪图杨铁心的妻子包惜弱的美色，杀害了郭靖的父亲郭啸天。'],
         ['杨铁心','包惜弱','配偶','杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
         ['张翠山','殷素素','配偶','张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。'],
         ['小龙女','杨过','师傅','小龙女是杨过的师父，与杨过互生情愫，但因师生恋不容于世。'],
         ['黄药师','黄蓉','父','黄药师，黄蓉之父，对其妻冯氏（小字阿衡）一往情深。'],
         ['郭啸天','郭靖','父','郭靖之父郭啸天和其义弟杨铁心因被段天德陷害，死于临安牛家村。']]

relation2idx = {'子女':0,'配偶':1,'师傅':2,'父':3}

lists, relation2idx

In [ ]:
datas, labels, pos_list1, pos_list2 = [], [], [], []
translation = 32
for entity1, entity2, relation, text in lists:
    # 找到第一个实体出现的下标
    idx1 = text.index(entity1)
    # 找到第二个实体出现的下标
    idx2 = text.index(entity2)
    sentence, pos1, pos2 = [], [], []
    for i, w in enumerate(text):
        sentence.append(w)
        # 计算句子中每个字与实体1首字的距离
        pos1.append(i-idx1+translation)
        # 计算句子中每个字与实体2首字的距离
        pos2.append(i-idx2+translation)
    datas.append(sentence)
    labels.append(relation2idx[relation])
    pos_list1.append(pos1)
    pos_list2.append(pos2)



In [ ]:
from collections import Counter
# 统计每个字出现的次数, sum(datas,[]) 的功能是将列表铺平
word_counts = Counter(sum(datas, []))
# 建立字典表，只记录出现次数不小于 2 的字
vocab = [w for w, f in iter(word_counts.items()) if f >= 2]
# word_counts, vocab


In [ ]:
# 构建词袋模型，和上一节实验相同，将字典从 2 开始编号，把 0 和 1 空出来，0 作为填充元素，1 作为不在字典中的字的编号
word2idx = dict((w,i+2) for i,w in enumerate(vocab))
# word2idx

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

# 构建输入，即对于样本中每一个字，从词袋模型中找到这个字对应的 idx，出现频率过低的字，并没有出现在词袋模型中，此时将这些字的 idx 取为 1
train_x = [[word2idx.get(w, 1) for w in s] for s in datas]

max_len = 64

# 在输入的左边填充 0
train_x = pad_sequences(train_x, max_len, value=0)
## 填充位置编码
train_pos1 = pad_sequences(pos_list1, max_len, value=0)
train_pos2 = pad_sequences(pos_list2, max_len, value=0)
# one_hot 编码 label
train_y = to_categorical(labels, num_classes=len(relation2idx))

train_x.shape, train_y.shape, train_pos1.shape, train_pos2.shape

In [ ]:
from keras.layers import Input, Embedding, concatenate, Conv1D, GlobalMaxPool1D, Dense, LSTM
from keras.models import Model

# 定义输入层
words = Input(shape=(max_len,),dtype='int32')
position1 = Input(shape=(max_len,),dtype='int32')
position2 = Input(shape=(max_len,),dtype='int32')
#  Embedding 层将输入进行编码
pos_emb1 = Embedding(output_dim=16, input_dim=256)(position1)
pos_emb2 = Embedding(output_dim=16, input_dim=256)(position2)
word_emb = Embedding(output_dim=16, input_dim=256)(words)
# 分别拼接 文本编码与位置1 和文本编码与位置2
concat1 = concatenate([word_emb, pos_emb1])
concat2 = concatenate([word_emb, pos_emb2])
# 卷积池化层
conv1 = Conv1D(filters=128, kernel_size=3)(concat1)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=3)(concat2)
pool2 = GlobalMaxPool1D()(conv2)
# 拼接，最后接全连接层，激活函数为 softmax
concat = concatenate([pool1, pool2])
out = Dense(units=len(relation2idx),activation='softmax')(concat)

model = Model(inputs=[words, position1, position2],outputs=out)
# 编译模型
model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 训练 50 次
model.fit([train_x, train_pos1, train_pos2], train_y, batch_size=8, epochs=50)
model.save('model_001.h5')

In [ ]:
test_instance = ['张翠山','殷素素','张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。']
test_ne1, test_ne2, test_text = test_instance
test_ne1, test_ne2, test_text

In [ ]:
# 将预测数据转换为向量
pred_x = [word2idx.get(w, 1) for w in test_text]
idx1 = test_text.index(test_ne1)
idx2 = test_text.index(test_ne2)
pos1 = [i-idx1+translation for i in range(len(test_text))]
pos2 = [i-idx2+translation for i in range(len(test_text))]
pred_x = pad_sequences([pred_x], max_len, value=0)
test_pos1 = pad_sequences([pos1], max_len, value=0)
test_pos2 = pad_sequences([pos2], max_len, value=0)
pred_x, test_pos1, test_pos2

In [ ]:
# 翻转 relation2idx 字典
idx2relation = dict(zip(relation2idx.values(),relation2idx.keys()))
# 使用模型进行预测
pred = model.predict([pred_x, test_pos1, test_pos2])
# 模型预测最大值的位置作为预测值
output_idx = np.argmax(pred)
# 找到 idx2relation 中实际的标签
output_label = idx2relation[output_idx]
pred, output_idx, output_label